In [51]:
import pandas as pd
import numpy as np

In [52]:
df=pd.read_csv('/content/drive/MyDrive/dataset/demand-forecasting-kernels-only/final_train.csv')

In [54]:
df.head()

,date,store,item,sales,day,year,month,datetime,weekday,weekday_name,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728
0,2013-01-01,1,1,2.639057,1,2013,1,2013-01-01,1,Tuesday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,1,1,2.484907,2,2013,1,2013-01-02,2,Wednesday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,1,1,2.708050,3,2013,1,2013-01-03,3,Thursday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,1,1,2.639057,4,2013,1,2013-01-04,4,Friday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,1,1,2.397895,5,2013,1,2013-01-05,5,Saturday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


here i have used describe just to get mean values for all shift/lag features 

In [55]:
df.describe()

,store,item,sales,day,year,month,weekday,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728
count,913000.000000,913000.000000,913000.000000,913000.000000,913000.000000,913000.000000,913000.000000,867500.000000,864000.000000,860500.000000,857000.000000,853500.000000,850000.000000,822000.000000,731000.000000,640000.000000,549000.000000
mean,5.500000,25.500000,3.821252,15.727820,2015.000548,6.523549,3.001643,52.129541,52.096150,52.053905,52.014013,51.977288,51.920541,51.269601,50.611449,49.466249,48.594454
std,2.872283,14.430878,0.576143,8.799329,1.414021,3.448535,1.999316,28.860266,28.850756,28.836125,28.819528,28.801229,28.783325,28.300877,27.875275,27.233630,26.644817
min,1.000000,1.000000,2.190948,1.000000,2013.000000,1.000000,0.000000,-1.080629,-2.994341,-2.686332,-2.185900,-1.879596,-1.680496,-0.766660,-1.077531,-1.061433,-1.132521
25%,3.000000,13.000000,3.433987,8.000000,2014.000000,4.000000,1.000000,29.554127,29.537595,29.518515,29.477917,29.477882,29.412214,29.132068,28.772473,28.125050,27.686138
50%,5.500000,25.500000,3.871201,16.000000,2015.000000,7.000000,3.000000,46.785380,46.753709,46.695217,46.655775,46.627580,46.584538,46.075308,45.513286,44.515972,43.804790
75%,8.000000,38.000000,4.262680,23.000000,2016.000000,10.000000,5.000000,69.454728,69.402893,69.327372,69.274253,69.214008,69.157253,68.324117,67.455616,65.907583,64.782432
max,10.000000,50.000000,5.446737,31.000000,2017.000000,12.000000,6.000000,230.847806,229.120986,231.706300,230.341126,231.499983,230.909596,232.455278,210.606059,196.625206,198.488047


In [56]:
data=df.copy()

Handled some missing values in newly generated features
(should be done in feature engineering)

In [57]:
df['sales_lag_91']=df['sales_lag_91'].fillna(df['sales_lag_91'].mean())

In [58]:
df['sales_lag_98']=df['sales_lag_98'].fillna(df['sales_lag_98'].mean())
df['sales_lag_105']=df['sales_lag_105'].fillna(df['sales_lag_105'].mean())
df['sales_lag_112']=df['sales_lag_112'].fillna(df['sales_lag_112'].mean())
df['sales_lag_119']=df['sales_lag_119'].fillna(df['sales_lag_119'].mean())
df['sales_lag_126']=df['sales_lag_126'].fillna(df['sales_lag_126'].mean())
df['sales_lag_182']=df['sales_lag_182'].fillna(df['sales_lag_182'].mean())
df['sales_lag_364']=df['sales_lag_364'].fillna(df['sales_lag_364'].mean())
df['sales_lag_546']=df['sales_lag_546'].fillna(df['sales_lag_546'].mean())
df['sales_lag_728']=df['sales_lag_728'].fillna(df['sales_lag_728'].mean())


In [59]:
df.head()

,date,store,item,sales,day,year,month,datetime,weekday,weekday_name,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728
0,2013-01-01,1,1,2.639057,1,2013,1,2013-01-01,1,Tuesday,52.129541,52.09615,52.053905,52.014013,51.977288,51.920541,51.269601,50.611449,49.466249,48.594454
1,2013-01-02,1,1,2.484907,2,2013,1,2013-01-02,2,Wednesday,52.129541,52.09615,52.053905,52.014013,51.977288,51.920541,51.269601,50.611449,49.466249,48.594454
2,2013-01-03,1,1,2.708050,3,2013,1,2013-01-03,3,Thursday,52.129541,52.09615,52.053905,52.014013,51.977288,51.920541,51.269601,50.611449,49.466249,48.594454
3,2013-01-04,1,1,2.639057,4,2013,1,2013-01-04,4,Friday,52.129541,52.09615,52.053905,52.014013,51.977288,51.920541,51.269601,50.611449,49.466249,48.594454
4,2013-01-05,1,1,2.397895,5,2013,1,2013-01-05,5,Saturday,52.129541,52.09615,52.053905,52.014013,51.977288,51.920541,51.269601,50.611449,49.466249,48.594454


took data before date 2017-01-01 as training and after this for testing

In [60]:
train = df.loc[(df["date"] < "2017-01-01"), :] # Until beginning of 2017
val = df.loc[(df["date"] >= "2017-01-01") & (df["date"] < "2017-04-01"), :] # First 3 months of 2017

cols = [col for col in train.columns if col not in ['date', 'id', "sales", "year"]]


In [61]:
cols=['store',
 'item',
 'day',
 'month',
 'weekday',
 'sales_lag_91',
 'sales_lag_98',
 'sales_lag_105',
 'sales_lag_112',
 'sales_lag_119',
 'sales_lag_126',
 'sales_lag_182',
 'sales_lag_364',
 'sales_lag_546',
 'sales_lag_728']

In [62]:
Y_train = train['sales']
X_train = train[cols]

Y_val = val['sales']
X_val = val[cols]

Y_train.shape, X_train.shape, Y_val.shape, X_val.shape

((730500,), (730500, 15), (45000,), (45000, 15))

LINEAR REGRESSION

In [63]:
from sklearn import linear_model

In [64]:
reg = linear_model.LinearRegression()

In [65]:
reg.fit(X_train,Y_train)

LinearRegression()

In [66]:
y_pred=reg.predict(X_val)

In [67]:
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val

In [68]:
smape(y_pred,Y_val)

5.700175637643756

Polynomial Regression

In [69]:
# importing libraries for polynomial transform
from sklearn.preprocessing import PolynomialFeatures
# for creating pipeline
from sklearn.pipeline import Pipeline
# creating pipeline and fitting it on data
Input=[('polynomial',PolynomialFeatures(degree=2)),('modal',linear_model.LinearRegression())]
pipe=Pipeline(Input)
pipe.fit(X_train,Y_train)

Pipeline(steps=[('polynomial', PolynomialFeatures()),
                ('modal', LinearRegression())])

In [70]:
y_pred=pipe.predict(X_val)

In [71]:
smape(y_pred,Y_val)

4.4003149872935206

Ridge regression

In [72]:
reg = linear_model.Ridge(alpha=.5)

In [73]:
reg.fit(X_train,Y_train)

Ridge(alpha=0.5)

In [74]:
y_pred=reg.predict(X_val)

In [75]:
smape(y_pred,Y_val)

5.700175627358081

Rigde Regression

In [76]:
reg = linear_model.RidgeCV(alphas=np.logspace(7,13))

In [77]:
reg.fit(X_train,Y_train)

RidgeCV(alphas=array([1.00000000e+07, 1.32571137e+07, 1.75751062e+07, 2.32995181e+07,
       3.08884360e+07, 4.09491506e+07, 5.42867544e+07, 7.19685673e+07,
       9.54095476e+07, 1.26485522e+08, 1.67683294e+08, 2.22299648e+08,
       2.94705170e+08, 3.90693994e+08, 5.17947468e+08, 6.86648845e+08,
       9.10298178e+08, 1.20679264e+09, 1.59985872e+09, 2.12095089e+09,
       2.81176870e+09, 3.72759372e+0...
       8.68511374e+09, 1.15139540e+10, 1.52641797e+10, 2.02358965e+10,
       2.68269580e+10, 3.55648031e+10, 4.71486636e+10, 6.25055193e+10,
       8.28642773e+10, 1.09854114e+11, 1.45634848e+11, 1.93069773e+11,
       2.55954792e+11, 3.39322177e+11, 4.49843267e+11, 5.96362332e+11,
       7.90604321e+11, 1.04811313e+12, 1.38949549e+12, 1.84206997e+12,
       2.44205309e+12, 3.23745754e+12, 4.29193426e+12, 5.68986603e+12,
       7.54312006e+12, 1.00000000e+13]))

In [78]:
y_pred=reg.predict(X_val)

In [79]:
smape(y_pred,Y_val)

5.614061715015793

In [80]:
from sklearn.linear_model import TweedieRegressor

In [81]:
reg = TweedieRegressor(power=1, alpha=0.5, link='log')

In [82]:
reg.fit(X_train,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_glm/glm.py:323: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


TweedieRegressor(alpha=0.5, link='log', power=1)

In [83]:
y_pred=reg.predict(X_val)

In [84]:
smape(y_pred,Y_val)

6.173757795676206

RandomForest Regressor

In [85]:
from sklearn.ensemble import RandomForestRegressor

In [86]:
regr = RandomForestRegressor(max_depth=4, random_state=45)

In [87]:
regr.fit(X_train,Y_train)

RandomForestRegressor(max_depth=4, random_state=45)

In [88]:
y_pred=regr.predict(X_val)

In [89]:
smape(y_pred,Y_val)

5.509917545828138

Light GBM regressor

In [90]:
import lightgbm as lgb

In [91]:

lgb_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'num_boost_round': 11000,
              'early_stopping_rounds': 200,
              'nthread': -1}

In [92]:
lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)

model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbval],
                  num_boost_round=lgb_params['num_boost_round'],
                  early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  verbose_eval=200)

y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)  

smape(np.expm1(y_pred_val), np.expm1(Y_val))

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[200]	training's l1: 0.167875	valid_1's l1: 0.15035
[400]	training's l1: 0.156352	valid_1's l1: 0.146644
[600]	training's l1: 0.151387	valid_1's l1: 0.143972
[800]	training's l1: 0.147831	valid_1's l1: 0.142042
[1000]	training's l1: 0.145189	valid_1's l1: 0.14066
[1200]	training's l1: 0.142876	valid_1's l1: 0.139688
[1400]	training's l1: 0.141362	valid_1's l1: 0.138891
[1600]	training's l1: 0.13997	valid_1's l1: 0.138148
[1800]	training's l1: 0.138611	valid_1's l1: 0.137542
[2000]	training's l1: 0.13756	valid_1's l1: 0.136976
[2200]	training's l1: 0.136698	valid_1's l1: 0.136551
[2400]	training's l1: 0.135901	valid_1's l1: 0.136184
[2600]	training's l1: 0.135101	valid_1's l1: 0.135871
[2800]	training's l1: 0.134394	valid_1's l1: 0.13556
[3000]	training's l1: 0.133753	valid_1's l1: 0.135308
[3200]	training's l1: 0.133299	valid_1's l1: 0.135082
[3400]	training's l1: 0.132812	valid_1's l1: 0.134849
[3600]	training's l1: 0.1324

13.633164371743927

getting the best iterations of the model

In [93]:
lgb_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'nthread': -1,
              "num_boost_round": model.best_iteration}

In [99]:
train.columns

Index(['date', 'store', 'item', 'sales', 'day', 'year', 'month', 'datetime',
       'weekday', 'weekday_name', 'sales_lag_91', 'sales_lag_98',
       'sales_lag_105', 'sales_lag_112', 'sales_lag_119', 'sales_lag_126',
       'sales_lag_182', 'sales_lag_364', 'sales_lag_546', 'sales_lag_728'],
      dtype='object')

loading test file and creating some lag/shift features

In [96]:
test=pd.read_csv('/content/drive/MyDrive/dataset/demand-forecasting-kernels-only/tested.csv')

In [97]:
test.head()

,id,date,store,item,day,year,month,datetime,weekday,weekday_name
0,0,2018-01-01,1,1,1,2018,1,2018-01-01,0,Monday
1,1,2018-01-02,1,1,2,2018,1,2018-01-02,1,Tuesday
2,2,2018-01-03,1,1,3,2018,1,2018-01-03,2,Wednesday
3,3,2018-01-04,1,1,4,2018,1,2018-01-04,3,Thursday
4,4,2018-01-05,1,1,5,2018,1,2018-01-05,4,Friday


In [100]:
test[['sales','sales_lag_91', 'sales_lag_98',
      'sales_lag_105', 'sales_lag_112',
      'sales_lag_119', 'sales_lag_126',
      'sales_lag_182', 'sales_lag_364', 
      'sales_lag_546', 'sales_lag_728']]=np.nan

In [101]:
test.head()

,id,date,store,item,day,year,month,datetime,weekday,weekday_name,sales,sales_lag_91,sales_lag_98,sales_lag_105,sales_lag_112,sales_lag_119,sales_lag_126,sales_lag_182,sales_lag_364,sales_lag_546,sales_lag_728
0,0,2018-01-01,1,1,1,2018,1,2018-01-01,0,Monday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-01-02,1,1,2,2018,1,2018-01-02,1,Tuesday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018-01-03,1,1,3,2018,1,2018-01-03,2,Wednesday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2018-01-04,1,1,4,2018,1,2018-01-04,3,Thursday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018-01-05,1,1,5,2018,1,2018-01-05,4,Friday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
train = df.loc[~df.sales.isna()]
Y_train = train['sales']
X_train = train[cols]

X_test = test[cols]

In [103]:
lgbtrain_all = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)

model = lgb.train(lgb_params, lgbtrain_all, num_boost_round=model.best_iteration)

test_preds = model.predict(X_test, num_iteration=model.best_iteration)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [104]:
submission_df = test.loc[:, ['id', 'sales']]
submission_df['sales'] = np.expm1(test_preds)
submission_df['id'] = submission_df.id.astype(int)

submission_df.to_csv('submission.csv', index=False)
submission_df.head(20)

,id,sales
0,0,10.861279
1,1,13.151382
2,2,13.281030
3,3,13.805656
4,4,14.970785
5,5,15.261705
6,6,15.837871
7,7,10.909946
8,8,13.093999
9,9,13.093472
